In [70]:
from connectPyspark import *
from pyspark.ml.feature import NGram
import pyspark.sql.functions as F

In [88]:
a = read_avro("output/10_MERGE_TAG")
a = a.repartition(60)

a.show(100, False)



# 🆂🅰🅼🅿🅻🅴

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [89]:
a= a.withColumn("sentence_id", F.monotonically_increasing_id())

In [90]:
a.show()

# 🆂🅰🅼🅿🅻🅴

+-------------------------------------+------------------------------+--------------------+--------+--------------------+-----------+
|                             original|                         token|                 tag|language|          normal_tag|sentence_id|
+-------------------------------------+------------------------------+--------------------+--------+--------------------+-----------+
| 经中央编办批准 以中国社会科学院历...|  [经, 中央, 编办, 批准, 以...|[n, n, v, v, p, n...| chinese|[NOUN, NOUN, VERB...|          0|
|      年月 中国天诚集团宣布收购俱乐部|  [年, 月, 中国, 天诚, 集团...|[m, m, ns, nr, n,...| chinese|[m, m, PROPN, PRO...|          1|
|是中国大陆唯一以政治领袖命名的路名...|  [是, 中国, 大陆, 唯一, 以...|[v, ns, n, b, p, ...| chinese|[VERB, PROPN, NOU...|          2|
| 另外 源自于中国的单字使用于正式而...|  [另外, 源自, 于, 中国, 的...|[c, v, p, ns, uj,...| chinese|[CONJ, VERB, ADP,...|          3|
| 然而 当时英方内部认为香港位处边陲...|  [然而, 当时, 英方, 内部, ...|[c, t, nz, f, v, ...| chinese|[CONJ, t, PROPN, ...|          4|
|  吴家翔 扁借幸妤拒搭长荣 暗批张荣...|  [吴家翔, 扁, 借, 幸妤, 拒...|[

In [9]:
## bigram

In [91]:
tokenNgram =NGram(n=2, inputCol="token", outputCol="token_gram")
tagNgram =NGram(n=2, inputCol="tag", outputCol="tag_gram")


In [92]:
n2 = tokenNgram.transform(a)
n2 = tagNgram.transform(n2)


In [93]:
n2.show()

# 🆂🅰🅼🅿🅻🅴 bigram

+-------------------------------------+------------------------------+--------------------+--------+--------------------+-----------+--------------------------------+--------------------+
|                             original|                         token|                 tag|language|          normal_tag|sentence_id|                      token_gram|            tag_gram|
+-------------------------------------+------------------------------+--------------------+--------+--------------------+-----------+--------------------------------+--------------------+
| 经中央编办批准 以中国社会科学院历...|  [经, 中央, 编办, 批准, 以...|[n, n, v, v, p, n...| chinese|[NOUN, NOUN, VERB...|          0|   [经 中央, 中央 编办, 编办 ...|[n n, n v, v v, v...|
|      年月 中国天诚集团宣布收购俱乐部|  [年, 月, 中国, 天诚, 集团...|[m, m, ns, nr, n,...| chinese|[m, m, PROPN, PRO...|          1|   [年 月, 月 中国, 中国 天诚...|[m m, m ns, ns nr...|
|是中国大陆唯一以政治领袖命名的路名...|  [是, 中国, 大陆, 唯一, 以...|[v, ns, n, b, p, ...| chinese|[VERB, PROPN, NOU...|          2|   [是 中国, 中国 大陆, 

In [94]:
from pyspark.sql.functions import arrays_zip, col, explode
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as F
t = n2.withColumn("tmp", arrays_zip("token_gram", "tag_gram"))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag","token", F.posexplode(col("tmp")))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag", "pos",col("col.token_gram"), col("col.tag_gram"))

t.show(100)


# 🆂🅰🅼🅿🅻🅴 bigram

+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+---------------------+--------+
|sentence_id|                             original|                 tag|language|                       token|          normal_tag|pos|           token_gram|tag_gram|
+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+---------------------+--------+
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  0|              经 中央|     n n|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  1|            中央 编办|     n v|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  2|            编办 批准|     v v|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 

In [95]:
write_avro(t, "ngramV2/N2_GRAM")

In [97]:
t.filter(t.language == "english").show()

# 🆂🅰🅼🅿🅻🅴 bigram

+-----------+--------------------+--------------------+--------+--------------------+----------+---+----------------+----------+
|sentence_id|            original|                 tag|language|               token|normal_tag|pos|      token_gram|  tag_gram|
+-----------+--------------------+--------------------+--------+--------------------+----------+---+----------------+----------+
|       5656|the college also ...|[DET, NOUN, ADV, ...| english|[the, college, al...|      null|  0|     the college|  DET NOUN|
|       5656|the college also ...|[DET, NOUN, ADV, ...| english|[the, college, al...|      null|  1|    college also|  NOUN ADV|
|       5656|the college also ...|[DET, NOUN, ADV, ...| english|[the, college, al...|      null|  2|        also has|  ADV VERB|
|       5656|the college also ...|[DET, NOUN, ADV, ...| english|[the, college, al...|      null|  3|           has a|  VERB DET|
|       5656|the college also ...|[DET, NOUN, ADV, ...| english|[the, college, al...|      null| 

In [98]:
## trigram

In [99]:
tokenNgram =NGram(n=3, inputCol="token", outputCol="token_gram")
tagNgram =NGram(n=3, inputCol="tag", outputCol="tag_gram")

In [100]:
n = tokenNgram.transform(a)
n = tagNgram.transform(n)

In [101]:
from pyspark.sql.functions import arrays_zip, col, explode
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as F
t = n.withColumn("tmp", arrays_zip("token_gram", "tag_gram"))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag","token", F.posexplode(col("tmp")))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag", "pos",col("col.token_gram"), col("col.tag_gram"))

t.show(100)

# 🆂🅰🅼🅿🅻🅴 trigram

+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+----------------------------+--------+
|sentence_id|                             original|                 tag|language|                       token|          normal_tag|pos|                  token_gram|tag_gram|
+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+----------------------------+--------+
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  0|                经 中央 编办|   n n v|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  1|              中央 编办 批准|   n v v|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  2|                编办 批准 以|   v v p|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v,

In [102]:
write_avro(t, "ngramV2/N3_GRAM")

In [ ]:
## 4gram

In [103]:
tokenNgram =NGram(n=4, inputCol="token", outputCol="token_gram")
tagNgram =NGram(n=4, inputCol="tag", outputCol="tag_gram")

In [104]:
n = tokenNgram.transform(a)
n = tagNgram.transform(n)

In [105]:
from pyspark.sql.functions import arrays_zip, col, explode
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as F
t = n.withColumn("tmp", arrays_zip("token_gram", "tag_gram"))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag","token", F.posexplode(col("tmp")))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag", "pos",col("col.token_gram"), col("col.tag_gram"))


# 🆂🅰🅼🅿🅻🅴 4gram
t.show(100)

+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+---------------------------------+----------+
|sentence_id|                             original|                 tag|language|                       token|          normal_tag|pos|                       token_gram|  tag_gram|
+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+---------------------------------+----------+
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  0|                经 中央 编办 批准|   n n v v|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  1|                中央 编办 批准 以|   n v v p|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  2|      编办 批准 以 中国社会科学院|  v v p nt|
|          0

In [106]:
write_avro(t, "ngramV2/N4_GRAM")

In [ ]:
# 5 gram

In [107]:
tokenNgram =NGram(n=5, inputCol="token", outputCol="token_gram")
tagNgram =NGram(n=5, inputCol="tag", outputCol="tag_gram")

In [108]:
n = tokenNgram.transform(a)
n = tagNgram.transform(n)

In [109]:
from pyspark.sql.functions import arrays_zip, col, explode
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as F
t = n.withColumn("tmp", arrays_zip("token_gram", "tag_gram"))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag","token", F.posexplode(col("tmp")))\
.select("sentence_id", "original", "tag","language", "token", "normal_tag", "pos",col("col.token_gram"), col("col.tag_gram"))


# 🆂🅰🅼🅿🅻🅴 5gram
t.show(100)

+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+----------------------------------+-------------+
|sentence_id|                             original|                 tag|language|                       token|          normal_tag|pos|                        token_gram|     tag_gram|
+-----------+-------------------------------------+--------------------+--------+----------------------------+--------------------+---+----------------------------------+-------------+
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  0|              经 中央 编办 批准 以|    n n v v p|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  1|  中央 编办 批准 以 中国社会科学院|   n v v p nt|
|          0| 经中央编办批准 以中国社会科学院历...|[n, n, v, v, p, n...| chinese|[经, 中央, 编办, 批准, 以...|[NOUN, NOUN, VERB...|  2|  编办 批准 以 中国社会科学院 历史|   v v p nt n

In [110]:
write_avro(t, "ngramV2/N5_GRAM")

In [111]:
g2 = read_avro("ngramV2/N2_GRAM")
g3 = read_avro("ngramV2/N3_GRAM")
g4 = read_avro("ngramV2/N4_GRAM")
g5 = read_avro("ngramV2/N5_GRAM")

In [112]:
g2 = g2.withColumn("gram_type", F.lit("2"))
g3 = g3.withColumn("gram_type", F.lit("3"))
g4 = g4.withColumn("gram_type", F.lit("4"))
g5 = g5.withColumn("gram_type", F.lit("5"))

In [113]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
merge = unionAll(g2, g3, g4, g5)

In [114]:
merge = merge.withColumn("unique_id", F.monotonically_increasing_id())

In [115]:
write_avro(merge, "output/11_ALL__NGRAM.avro")

In [ ]:
### 24 RETAIN, KEYWORDS N-GRAM

In [116]:
import pyspark.sql.types as T

In [117]:
engKey = ["university","school","college"]
chiKey = ["中国","台湾","香港"]
bmKey = ["kampung","sekolah","daerah","barat","utara","komun"]

keywords = set(engKey).union(set(chiKey)).union(set(bmKey))

def contains_keywords(text):
    for x in text.split(" "):
        if x in keywords:
            return True
        else:
            return False

keyDeter = udf(lambda x : contains_keywords(x), T.BooleanType())

In [119]:
df = read_avro("output/11_ALL__NGRAM.avro")
withKey = df\
.withColumn("containsKey", keyDeter(F.col("token_gram"))) \
.filter(F.col("containsKey") == True)

# 🆂🅰🅼🅿🅻🅴 ngram with key only
withKey.show(100, False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------

In [120]:
write_avro(withKey, "output/12_KEY_ONLY")

In [ ]:
## is middle 

In [121]:
import pyspark.sql.functions as f
import pyspark.sql.types as T


engKey = ["university","school","college"]
chiKey = ["中国","台湾","香港"]
bmKey = ["kampung","sekolah","daerah","barat","utara","komun"]

keywords = set(engKey).union(set(chiKey)).union(set(bmKey))

def is_middle(gramType, token_gram):
    token_gram = token_gram.split(" ")
    if gramType == "2" or gramType == "4":
        return False
    elif gramType == "3":
        if token_gram[1] in keywords:
            return True
        else:
            return False
    elif gramType == "5":
        if token_gram[2] in keywords:
            return True
        else:
            return False
    else:
        return False
    
middleUDF = f.udf(lambda x,y : is_middle(x,y), T.BooleanType() )

In [122]:
df = read_avro("output/12_KEY_ONLY")

In [123]:
df = df.withColumn("middle_key", middleUDF(df.gram_type, df.token_gram))

In [124]:
shutil.rmtree("output/25_CENTER_DETERMINED")

write_avro(df, "output/25_CENTER_DETERMINED")

In [125]:
df = read_avro( "output/25_CENTER_DETERMINED")

In [127]:
df.filter((df.language == "english") ).show()

+-----------+--------------------+--------------------+--------+--------------------+----------+---+-------------------+----------+---------+-----------+-----------+----------+
|sentence_id|            original|                 tag|language|               token|normal_tag|pos|         token_gram|  tag_gram|gram_type|  unique_id|containsKey|middle_key|
+-----------+--------------------+--------------------+--------+--------------------+----------+---+-------------------+----------+---------+-----------+-----------+----------+
|51539613210|successful comple...|[ADJ, NOUN, ADP, ...| english|[successful, comp...|      null| 16|      school oabcig| NOUN NOUN|        2|17180004394|       true|     false|
|51539613210|successful comple...|[ADJ, NOUN, ADP, ...| english|[successful, comp...|      null| 19|          school or|NOUN CCONJ|        2|17180004397|       true|     false|
|51539613210|successful comple...|[ADJ, NOUN, ADP, ...| english|[successful, comp...|      null| 24|school respecti

In [128]:
df.filter((df.language == "malay") ).show()

+-----------+--------------------+--------------------+--------+--------------------+----------+---+----------------+-----------+---------+-----------+-----------+----------+
|sentence_id|            original|                 tag|language|               token|normal_tag|pos|      token_gram|   tag_gram|gram_type|  unique_id|containsKey|middle_key|
+-----------+--------------------+--------------------+--------+--------------------+----------+---+----------------+-----------+---------+-----------+-----------+----------+
|51539619243|leonessa merupaka...|[PROPN, VERB, DET...|   malay|[leonessa, merupa...|      null|  3|       komun dan| NOUN CCONJ|        2|17180144955|       true|     false|
|51539619245|rawang pasar vi m...|[NOUN, NOUN, PROP...|   malay|[rawang, pasar, v...|      null| 18| utara indonesia|PROPN PROPN|        2|17180144990|       true|     false|
|51539619247|terdapat buah des...|[VERB, NOUN, NOUN...|   malay|[terdapat, buah, ...|      null|  5|daerah kecamatan| NOUN PR